# Submission Information:

### Team Name: AllTheBest

### Team Member 1:
* UNI: **pw2435**
* Name: **Pingyuan Wang**

### Team Member 2:
* UNI:  **cz2431**
* Name: **Chenchao Zang**

# Step0 - Import Libraries, Load Data [0 points]

This is the basic step where you can load the data and create train and test sets for internal validation as per your convinience.

#### Step 0.1 Import Packages

In [1]:
#write code below, you can make multiple cells
import os
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.rcParams["figure.dpi"] = 100
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn import preprocessing

#### Step 0.2 Load Data

In [2]:
with open(os.path.abspath(os.path.join("data","data.csv"))) as file:
    df = pd.read_csv(file)

# Step1 - Exploration and Preparation [10 points]

In this step, we expect you to look into the data and try to understand it before modeling. This understanding may lead to some basic data preparation steps which are common across the two model sets required.

#### Step 1.1 Data Cleaning

In [3]:
def get_label_feature(df):
    # Get the Label
    y = df['subscribed']
    lb = preprocessing.LabelBinarizer(neg_label=-1, pos_label=1)
    y = lb.fit_transform(y)
    y = y.ravel()
    # Drop duration columns
    X = df.drop('duration', 1)
    X = X.drop('credit_default', 1)
    X = df.drop('subscribed', 1)
    X = X.drop('euribor3m',1)
    X = X.drop('emp_var_rate',1)
    return X,y

In [4]:
def processing_holdout(holdout):
    # Drop duration columns
    holdout = holdout.drop('duration', 1)
    holdout = holdout.drop('ID', 1) 
    holdout = holdout.drop('credit_default', 1)
    holdout = holdout.drop('euribor3m',1)
    holdout = holdout.drop('emp_var_rate',1)
    return holdout

In [5]:
def encode_label(X):
    binarizer = preprocessing.Binarizer(threshold=999).fit(X['prev_days'])
    X['prev_days'] = binarizer.transform(X['prev_days']).T
    return X

In [6]:
def replace_unknown(X):
    for i in range(1,7):
        X.iloc[:, i].replace(to_replace="unknown", value=np.nan, inplace=True)
        X.loc[:]['prev_days'].replace(to_replace=999, value=-1, inplace=True)
    # Feature where NA = "nonexistent"
    X.iloc[:, 13].replace(to_replace="nonexistent", value=np.nan, inplace=True)
    return X

In [7]:
def get_dummy(X_train):
    dum_cat = ['job', 'marital_status','education',
            'housing', 'loan', 'contact', 'month', 'day_of_week',
            'prev_outcomes']
    num_col=['age', 'campaign','cons_price_idx', 'cons_conf_idx', 
         'nr_employed','prev_contacts','prev_days']
    
    x_train_dummy = pd.DataFrame()
    for ele in (dum_cat):
        x_train_dummy = pd.concat([x_train_dummy,pd.get_dummies(X_train[ele], prefix=ele)],axis = 1)
    
    x_num_train = X_train[num_col]
    x_train_dummy = pd.concat([x_train_dummy,x_num_train],axis=1)
    return x_train_dummy

In [8]:
X, y = get_label_feature(df)
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
X_train = replace_unknown(X_train)
X_train = get_dummy(X_train)

/Users/chenchaozang/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [10]:
X_test = replace_unknown(X_test)
X_test = get_dummy(X_test)

/Users/chenchaozang/anaconda/lib/python3.5/site-packages/pandas/core/generic.py:3443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
X_train.shape, X_test.shape

((24712, 52), (8238, 52))

# Step2 - ModelSet1 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set1:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. Any classification algorithm covered in class apart from tree-based models can be tested here.

### Step 2.1 Models

#### LogisticRegression

In [12]:
skfold = StratifiedKFold(n_splits=5,shuffle=True)
lr = LogisticRegression(C = 100)
np.mean(cross_val_score(lr,X_train,y_train,cv=skfold,scoring='roc_auc'))

0.76966767458053709

In [13]:
lr.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
from sklearn.metrics import roc_auc_score
test_y = lr.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.75523898573809689

In [15]:
sum(y_test),sum(lr.predict(X_test))

(-6388, -7690)

#### SVM

In [606]:
# clf = svm.SVC(probability=True)
# clf.fit(X_train,y_train)
# np.mean(cross_val_score(clf,X_train,y_train,cv=skfold,scoring='roc_auc'))

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [607]:
# test_y = clf.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,test_y)

0.70839147481234355

#### Weighted SVM

In [608]:
# wclf = svm.SVC(kernel='linear', class_weight={1: 5},probability=True)
# wclf.fit(X_train,y_train)
# np.mean(cross_val_score(wclf,X_train,y_train,cv=skfold,scoring='roc_auc'))

array([ 0.777,  0.793,  0.797,  0.805,  0.779])

In [609]:
# test_y = wclf.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,test_y)

0.7951122265090127

#### Linear Regression

In [16]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train,y_train)
np.mean(cross_val_score(reg,X_train,y_train,cv=skfold,scoring='roc_auc'))

0.79184512249612737

In [617]:
# test_y = reg.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,test_y)

#### ElasticNet

In [721]:
from sklearn.linear_model import ElasticNet
enet = ElasticNet()
enet.fit(X_train,y_train)
np.mean(cross_val_score(enet,X_train,y_train,cv=skfold,scoring='roc_auc'))

0.75160846504823997

In [619]:
# test_y = enet.predict_proba(X_test)[:,1]
# roc_auc_score(y_test,test_y)

#### LDA

In [841]:
from sklearn.lda import LDA
est = LDA()
est.fit(X_train,y_train)
np.mean(cross_val_score(est,X_train,y_train,cv=skfold,scoring='roc_auc'))

/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variables are collinear.
  warnings.warn("Variables are collinear.")
/Users/chenchaozang/anaconda/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:387: UserWarning: Variabl

0.78740579930836074

In [842]:
test_y = est.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.78658051892464798

### Feature Selection

In [736]:
# Look at correlations between features
# import seaborn as sns
# Get to the numeric columns by inversion            
num_list = X_train.columns 

# Create Dataframe containing only numerical features
df_num = X_train[num_list]
df_num.astype(float).corr()

c = df_num.astype(float).corr().abs()

s = c.unstack()
so = s.order(kind="quicksort")
# [2940:2968]
print (so[2841:2859])
# f, ax = plt.subplots(figsize=(11, 11))
# plt.title('Pearson Correlation of features')

# # Draw the heatmap using seaborn
# sns.heatmap(df_num.astype(float).corr(),linewidths=0.25,vmax=1.0, square=True, cmap="cubehelix", linecolor='k', annot=True)

education_professional.course  job_technician            0.480725
prev_contacts                  nr_employed               0.501015
nr_employed                    prev_contacts             0.501015
prev_contacts                  prev_days                 0.509249
prev_days                      prev_contacts             0.509249
prev_outcomes_success          prev_contacts             0.532966
prev_contacts                  prev_outcomes_success     0.532966
prev_outcomes_failure          prev_contacts             0.674285
prev_contacts                  prev_outcomes_failure     0.674285
marital_status_married         marital_status_single     0.773760
marital_status_single          marital_status_married    0.773760
prev_outcomes_success          prev_days                 0.781928
prev_days                      prev_outcomes_success     0.781928
nr_employed                    emp_var_rate              0.902315
emp_var_rate                   nr_employed               0.902315
loan_yes  

/Users/chenchaozang/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: FutureWarning: order is deprecated, use sort_values(...)


### Feature Selection
As we see LDA indicates that there exists multicollinearity problem in our features, we decide to do a feature selection based on removing dependent features on the result of variance threshold. Basically remove features that has low variance. However, after we remove those features and retrain our model, all the performance (cross_val_score and test roc_auc) becomse worse. Thus, we decide just to keep those variables.



In [ ]:
# Remove collinear features
# sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
# mmm=sel.fit(x_train_dummy)
# new_x=mmm.transform(x_train_dummy)
# xxx=mmm.transform(x_test_dummy)
# cross_val_score(lda,new_x,y_train,cv=skfold)
# #Results becomes worse after feature selection for all the following models.
# lr.fit(new_x,y_train)
# pred_prob=lr.predict_proba(xxx)[:,1]
# roc_auc_score(y_test,pred_prob

# Step3 - ModelSet2 [35 points]

In this step, we expect you to perform the following steps relevant to the models you choose for set2:

* feature engineering
* validation
* feature selection
* final model selection

You may select up to 5 models in this step for the purpose of final ensemble. We encourage you to try decition tree, random forest and gradient boosted tree methods here and pick the one which you think works best.

#### Randome Forest (Overfitting)

In [843]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
np.mean(cross_val_score(rfc,X_train,y_train,cv=skfold))

0.89264335225129376

In [844]:
test_y = rfc.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.74844308262770232

#### Adaboost 

In [18]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()
clf.fit(X_train,y_train)
np.mean(cross_val_score(clf,X_train,y_train,cv=skfold))

0.89741822457930565

In [846]:
test_y = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.78312690939313856

#### Gradient Tree Boosting

In [19]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier()
clf.fit(X_train, y_train)
np.mean(cross_val_score(clf,X_train,y_train,cv=skfold))

0.89867264639635669

In [848]:
test_y = clf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.79175549803314105

### Feature selection based on feature importance
We tried feature selection based on feature importance, however, the result again becomes worse for all the models. Thus, we decide to keep all the features.


In [22]:
#Try feature selection based on importance, the result becomse worse. From 77-72
# rfc.fit(x_train_dummy,y_train)
# imp=rfc.feature_importances_
# from sklearn.feature_selection import SelectFromModel
# model = SelectFromModel(rfc, prefit=True)
# X_new = model.transform(x_train_dummy)
# x_test_new=model.transform(x_test_dummy)
# one_hot_new=model.transform(onehot_final)
# rfc=RandomForestClassifier()
# cross_val_score(rfc,X_new,y_train,cv=skfold)
# rfc.fit(X_new,y_train)
# pred_prob=rfc.predict_proba(x_test_new)[:,1]
# roc_auc_score(y_test,pred_prob)

### Resampling
We also notice that the response variable is very imbalanced, thus we decide to resampling the data to retrain the model. We used undersampling and oversampling, however, the cross_val_score becomes better, but the public score board on Kaggle becomes much worse, the score around 0.5. Thus, we decide not do resampling.

In [ ]:
#Random Under Sampling, however, the results becomes worse
# rus=RandomUnderSampler()
# aaa,bbb=rus.fit_sample(x_train_dummy,y_train)
#Random Over Sampling, however, the results becomes worse
# ros=RandomOverSampler()
# aaa,bbb=ros.fit_sample(x_train_dummy,y_train)

# Step4 - Ensemble [20 points + 10 Bonus points]

In this step, we expect you to use the models created before and create new predictions. You should definitely try poor man's stacking but we encourage you to think of different ensemble techniques as well. We will judge your creativity and improvement in model performance using ensemble models and you can potentially earn 10 bonus points here.

In [20]:
from sklearn.ensemble import VotingClassifier
# Training classifiers
clf1 = AdaBoostClassifier()
clf2 = GradientBoostingClassifier()
clf3 = svm.SVC(kernel='linear', class_weight={1: 5},probability=True)
# eclf = VotingClassifier(estimators=[('adaboost', clf1), ('gdb', clf2), ('svc', clf3)], voting='soft', weights=[1,2,2])
eclf = VotingClassifier(estimators=[('adaboost', clf1), ('gdb', clf2)], voting='soft', weights=[1,2])
clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)
clf3 = clf3.fit(X_train, y_train)
eclf = eclf.fit(X_train, y_train)

In [21]:
test_y = eclf.predict_proba(X_test)[:,1]
roc_auc_score(y_test,test_y)

0.78091499403136211

## Holdout Part

In [871]:
# hold_out = pd.read_csv('/Users/chenchaozang/Downloads/holdout.csv',sep=',')

In [872]:
# # Holdout
# holdout = processing_holdout(hold_out)
# holdout = replace_unknown(holdout)
# holdout = get_dummy(holdout)

In [873]:
# # Write results to csv
# res = eclf.predict_proba(holdout)[:,1]
# final_result = pd.concat([hold_out['ID'],pd.DataFrame(res)], axis=1)
# final_result.columns = ["ID","subscribed"]
# header = ["ID","subscribed"]
# final_result.to_csv("holdout_result.csv", columns = header, index = False)

In [ ]:
#write code below, you can make multiple cells
def test_accuracy():  
    assert roc_auc_score(y_test,test_y)> 0.78
test_accuracy()